# 1. 기존 방법 비판
**LSA**는 전체 통계 정보 반영, 단어 의미 유추에 어려움   
**Word2Vec**은 단어 의미 유추, 전체 통계 정보 반영 어려움   
**GloVe**(Global Vectors for Word Representation)는 카운트와 예측 기반 모두 사용

# 2. Window based Co-Occurence Matrix
전체 단어 집합으로 윈도우 크기에 따라 행렬 구성   

윈도우 크기 1일 때,   
문장1: I like deep learning   
문장2: I like NLP   
문장3: I enjoy flying   

카운트|I|like|enjoy|deep|learning|NLP|flying
:--|:--|:--|:--|:--|:--|:--|:--
I|0|2|1|0|0|0|0
like|2|0|0|1|0|1|0
enjoy|1|0|0|0|0|0|1
deep|0|1|0|0|1|0|0
learning|0|0|0|1|0|0|0
NLP|0|1|0|0|0|0|0
flying|0|0|1|0|0|0|0

# 3. Co-Occurence Probability
동시 등장 행렬에서 특정 단어 전체 등장 횟수와, 등장시 어떤 단어가 등장한 횟수의 조건부 확률

동시 등장 확률과 크기 관계 비|k=solid|k=gas|k=water|k=fashion
:--|:--|:--|:--|:--
P(ice 등장시 k 등장 확률)|큰 값|작은 값|큰 값|작은 값
P(steam 등장시 k 등장 확률)|작은 값|큰 값|큰 값|작은 값
P(ice 등장시 k 등장 확률) / P(steam 등장시 k 등장 확률)|큰 값|작은 값|1에 가까움|1에 가까움

# 4. Loss function
동시 등장 확률은 중심 단어와 주변 단어 임베딩 벡터 내적   
(P는 중심 단어 i 등장시 윈도우 내 주변 단어 k 등장 확률)
$$w_{i} \cdot \tilde{w_{k}} \approx P(k\,|\,i) = P_{ik}$$

함수 F에 벡터 적용시, 동시 등장 확률 비가 나오는 초기 식 가정
$$F(w_{i}, w_{j}, \tilde{w_{k}}) = \frac{P_{ik}}{P_{jk}}$$

1. F는 동시 등장 확률 비를 벡터 공간에 인코딩하는 목적
$$F(w_{i} - w_{j}, \tilde{w_{k}}) = \frac{P_{ik}}{P_{jk}}$$

2. 좌변은 벡터고 우변은 스칼라임을 성립해주는 내적   
(선형 공간에서 단어 의미 관계를 표현하고자 뺄셈과 내적 사용)
$$F((w_{i} - w_{j})^{T}\tilde{w_{k}}) = \frac{P_{ik}}{P_{jk}}$$   

3. 이때 F는 준동형(Homomorphism) 만족   
$$F(a + b) = F(a)F(b),\,F(a - b) = \frac{F(a)}{F(b)},\,\forall{a,b} \in \mathbb{R}$$

적용시
$$F((w_{i} - w_{j})^{T}\tilde{w_{k}}) = \frac{F(w_{i}^{T}\tilde{w_{k}})}{F(w_{j}^{T}\tilde{w_{k}})} = \frac{P_{ik}}{P_{jk}}$$

이를 만족하는 F는 Exponential function   
(X는 중심 단어 i 등장시 윈도우 내 주변 단어 k 등장 횟수)
$$exp(w_{i}^{T}\tilde{w_{k}} - w_{j}^{T}\tilde{w_{k}}) = \frac{exp(w_{i}^T\tilde{w_{k}})}{exp(w_{j}^T\tilde{w_{k}})} = \frac{P_{ik}}{P_{jk}}$$
$$exp(w_{i}^{T}\tilde{w_{k}}) = P_{ik} = \frac{X_{ik}}{X_{i}}$$
$$w_{i}^{T}\tilde{w_{k}} = logP_{ik} = log(\frac{X_{ik}}{X_{i}}) = logX_{ik} - logX_{i}$$

각 임베딩 벡터 위치를 바꾸어도 식이 성립하고자, 로그항을 편향으로 대체
$$w_{i}^{T}\tilde{w_{k}} + b_{i} + \tilde{b_{k}} = logX_{ik}$$

손실 함수는 이 차이를 최소화하는 일반화 식
$$Loss function = \sum_{m,n=1}^{V}(w_{m}^{T} + b_{m} + \tilde{b_{n}} - logX_{mn})^{2}$$
   
희소 행렬에서 낮은 빈도수에 가중치를 주는 weight function 도입, X가 클 때 지나친 가중치를 막고자 최댓값 설정   
$$f(x) = min(1, (\frac{x}{x_{max}})^{\frac{3}{4}})$$
![weight function](https://wikidocs.net/images/page/22885/%EA%B0%80%EC%A4%91%EC%B9%98.PNG "weight function")

최종 손실 함수
$$Loss function = \sum_{m,n=1}^{V}\,f(X_{mn})(w_{m}^{T}\tilde{w_{n}} + b_{m} + \tilde{b_{n}} - logX_{mn})^{2}$$

# 5. GloVe 훈련

In [1]:
import nltk
nltk.download('punkt')

import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml",
                           filename="ted_en-20160408.xml")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1e5cecf0e20>)

In [2]:
# 전처리
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

parse_text = '\n'.join(target_text.xpath('//content/text()'))
content_text = re.sub(r'\([^)]*\)', '', parse_text)

sent_text = sent_tokenize(content_text)

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

result = [word_tokenize(sentence) for sentence in normalized_text]

In [3]:
from glove import Corpus, Glove

# 동시 등장 행렬 생성
corpus = Corpus()
corpus.fit(result, window=5)

glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [4]:
model_result1 = glove.most_similar("man")
print(model_result1)

[('woman', 0.9648368487300606), ('guy', 0.87853441458273), ('girl', 0.8534987437931901), ('young', 0.846071380470317)]


In [5]:
model_result2 = glove.most_similar("boy")
print(model_result2)

[('girl', 0.9476923851537493), ('woman', 0.8419994258815078), ('kid', 0.8361017051820097), ('man', 0.8279171755525907)]


In [6]:
model_result3 = glove.most_similar("university")
print(model_result3)

[('harvard', 0.888141053853674), ('mit', 0.8571900561051062), ('cambridge', 0.8353162335157012), ('stanford', 0.8343890928881722)]


In [7]:
model_result4 = glove.most_similar("water")
print(model_result4)

[('clean', 0.8436805363087417), ('air', 0.8276533018109319), ('electricity', 0.818030617567804), ('fresh', 0.8173633565603985)]


In [8]:
model_result5 = glove.most_similar("physics")
print(model_result5)

[('chemistry', 0.8938172020985033), ('economics', 0.8825295341379408), ('beauty', 0.8640229385013825), ('mathematics', 0.8553610296644241)]


In [9]:
model_result6 = glove.most_similar("muscle")
print(model_result6)

[('tissue', 0.8488022951766478), ('nerve', 0.836366071812534), ('stem', 0.7888055281526625), ('channel', 0.7717473953205604)]


In [10]:
model_result7 = glove.most_similar("clean")
print(model_result7)

[('water', 0.8436805363087417), ('fresh', 0.8321380109071955), ('wind', 0.8010232560853364), ('heat', 0.7796518615106788)]
